In [ ]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1v7mZebFdVHJ__-q6T2agoSWZOto_kZZZ --output stock_data.xlsx
!gdown --id 1ugJJFEEqCttE9lKjX-TYRsb3gHGCxHDm --output bda2022_mid_news_2019.csv
!gdown --id 1lJa5jjLU-O9B__LA0D0aC2kZqDD7-DMs --output bda2022_mid_news_2020.csv
!gdown --id 1PCqcelrk5VqcCQciaUBocdmAUbDNOPnu --output bda2022_mid_news_2021.csv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=8874df33b17999b7dad58af823663f6ad42370fa3078cb263974c994b483b8cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_iaay5i/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1v7mZebFdVHJ__-q6T2agoSWZOto_kZZZ
To: /content/stock_data.xlsx
100% 500M/500M [00:05<00:00, 98.2MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.

In [ ]:
import pandas as pd
import numpy as np

#turn csv to dataframe
bda2022_mid_news_2019 = pd.read_csv("bda2022_mid_news_2019.csv")
bda2022_mid_news_2020 = pd.read_csv("bda2022_mid_news_2020.csv")
bda2022_mid_news_2021 = pd.read_csv("bda2022_mid_news_2021.csv")

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")
path="/content/drive/MyDrive/110-2 big data/database"
os.chdir(path)

Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
import os
from tqdm import tqdm
from collections import Counter
import json
import math
import re
import datetime

In [ ]:
len(bda2022_mid_news_2020)

157045

In [ ]:
def checkfinance(content):
    finance_related = ['彰銀','京城銀','台中銀','旺旺保','華票','台產','臺企銀','高雄銀','高雄銀甲特','聯邦銀','聯邦銀甲特','遠東銀','安泰銀','新產','中再保','第一保','統一證','三商壽','華南金','富邦金','富邦特','國泰金','國泰特','開發金','玉山金','元大金','兆豐金','台新金','台新戊特','新光金','國票金','永豐金','中信金','第一金','王道銀','上海商銀','合庫金','群益證','群益期']
    for i in range(len(finance_related)):
        if finance_related[i] in content:
            return True
        else:
            continue
    return False

In [ ]:
base_pth = os.getcwd()
pros_pth = os.path.join(base_pth, 'pros')

finance_related = ['彰銀', '京城銀', '台中銀', '旺旺保', '華票', '台產', '臺企銀', '高雄銀', '高雄銀甲特', '聯邦銀', '聯邦銀甲特', '遠東銀', \
                   '安泰銀', '新產', '中再保', '第一保', '統一證', '三商壽', '華南金', '富邦金', '富邦特', '國泰金', '國泰特', '開發金', '玉山金', \
                   '元大金', '兆豐金', '台新金', '台新戊特', '新光金', '國票金', '永豐金', '中信金', '第一金', '王道銀', '上海商銀', '合庫金', '群益證', '群益期']

strick_bound = ['金融', '金控']
up_kwd = ['漲', '反彈', '揚', '買壓']
down_kwd = ['跌', '失守', '觀望', '保守', '緩', '賣壓', '挫']
mutual_forb = ['震盪', '止', '抗', '整理']


In [ ]:
def csv_to_df(path):
    return pd.read_csv(path)
def df_to_csv(df, path):
    df.to_csv(path, encoding='utf_8_sig')


In [ ]:
def get_news():
    news2019 = csv_to_df('bda2022_mid_news_2019.csv')
    news2020 = csv_to_df('bda2022_mid_news_2020.csv')
    news2021 = csv_to_df('bda2022_mid_news_2021.csv')
    news = pd.concat([news2019, news2020, news2021])
    return news


def check(title, content, keywords, prohibits, cons_tl=True, cons_ct=False):
    string = ''
    if cons_tl: string += title
    if cons_ct and isinstance(content, str): string += content

    for i in keywords:
        if i in string:
            if prohibits is not None:
                for j in prohibits:
                    if j in string:
                        return False
            return True
        else:
            continue
    return False


def news_filter(news, kws, prohs):
    filtered = []
    for idx, line in tqdm(news.iterrows()):
        # string = line['title']
        # if isinstance(line['content'], str): string+= line['content']
        # if checkfinance(string): filtered.append(line)
        if check(line['title'], line['content'], kws, prohs): filtered.append(line)

    if len(filtered) != 0:
        ret_news = pd.concat(filtered, axis=1)
        ret_news = ret_news.T
        return ret_news
    else:
        return None

def gen_filtered_news():
    news = get_news()
    news = news_filter(news, finance_related, None)
    news.to_csv('filtered_0.csv', encoding='utf_8_sig')
    news = news_filter(news, strick_bound, None)
    news.to_csv('filtered_1.csv', encoding='utf_8_sig')
    up_news = news_filter(news, up_kwd, down_kwd + mutual_forb)
    down_news = news_filter(news, down_kwd, up_kwd + mutual_forb)
    up_news.to_csv('filtered_up.csv', encoding='utf_8_sig')
    down_news.to_csv('filtered_down.csv', encoding='utf_8_sig')

gen_filtered_news()

587392it [00:36, 15998.15it/s]
14420it [00:00, 17070.90it/s]
4698it [00:00, 17524.82it/s]
4698it [00:00, 16712.13it/s]


In [ ]:
def phraselst_to_json(phraselist):
    with open(os.path.join(pros_pth, 'phraselist_{}.json'.format(datetime.now().strftime('%m%d%H%M'))), 'w') as file:
        json.dump(phraselist, file, default=lambda o: o.__dict__, sort_keys=True, indent=4)
        print('Save Phrase List Complete')


def json_to_phraselst(filename='phraselist.json'):
    path = os.path.join(pros_pth, filename)
    with open(path, 'r') as file:
        dictlist = json.load(file)
    ret_list = []
    for elm in dictlist:
        ret_list.append(Phrase(**elm))
    print('Load Phrase List Complete')
    return ret_list

In [ ]:
def gen_freqs(sub_news):
    tf_ctr = Counter()
    df_ctr = Counter()
    slices_lst = []
    for idx, line in tqdm(sub_news.iterrows()):
        string = line['title']
        if isinstance(line['content'], str): string += line['content']
        slices = split_sentence_to_phrase(string)
        tf_ctr += Counter(slices)
        df_ctr += Counter(set(slices))
        slices_lst.append(slices)

    sub_news['slices'] = slices_lst
    return sub_news, tf_ctr, df_ctr
    # for idx, line in tqdm(sub_news.iterrows):
    #    tf = Counter(slices)
    #    df = Counter(set(slices))


class Phrase:
    def __init__(self, **kwargs):
        self.name = None
        self.tf_up = None
        self.df_up = None
        self.tf_down = None
        self.df_down = None
        self.N_up = None
        self.N_down = None

        self.__dict__.update(kwargs)
        self.tf_all = self.tf_up + self.tf_down
        self.df_all = self.df_up + self.df_down

        self.N_ttl = self.N_up + self.N_down
        for elm in ['up', 'down']:
            self.calc_tfidf(elm)

    def __str__(self):
        frqstr = 'Phrase:{}\nUP: tf={}, df={}, tfidf={}/ {}\nDN: tf={}, df={}, tfidf={} / \nALL: tf={}, df={} / {}\n' \
            .format(self.name, self.tf_up, self.df_up, self.tfidf_up, self.N_up,
                    self.tf_down, self.df_down, self.tfidf_down, self.N_down,
                    self.tf_all, self.df_all, self.N_ttl)

        return frqstr + '\n' + '-' * 30

    def calc_tfidf(self, lmttyp):
        # tf-idf = (1+log(tf)) * log(N_ttl/df) ???
        N_tmp = self.N_up if lmttyp == 'up' else self.N_down
        if lmttyp == 'up':
            self.tfidf_up = (1 + math.log(self.tf_up + 1e-4)) * math.log(self.N_ttl / self.df_all)
        elif lmttyp == 'down':
            self.tfidf_down = (1 + math.log(self.tf_down + 1e-4)) * math.log(self.N_ttl / self.df_all)


def gen_phrase_lst(tf_up_ctr, df_up_ctr, tf_down_ctr, df_down_ctr, N_up, N_down):
    phraselst = []
    df_all_ctr = df_up_ctr + df_down_ctr
    for name, times in df_all_ctr.most_common():
        phraselst.append(Phrase(name=name,
                                 tf_up=tf_up_ctr[name] if name in tf_up_ctr else 0,
                                 df_up=df_up_ctr[name] if name in df_up_ctr else 0,
                                 tf_down=tf_down_ctr[name] if name in tf_down_ctr else 0,
                                 df_down=df_down_ctr[name] if name in df_down_ctr else 0,
                                 N_up=N_up,
                                 N_down=N_down))
    return phraselst

def adding_ctr():
    up_news = csv_to_df(os.path.join(pros_pth, 'filtered_up.csv'))
    down_news = csv_to_df(os.path.join(pros_pth, 'filtered_down.csv'))
    up_news, tf_up_ctr, df_up_ctr = gen_freqs(up_news)
    down_news, tf_down_ctr, df_down_ctr = gen_freqs(down_news)
    phraselst = gen_phrase_lst(tf_up_ctr, df_up_ctr, tf_down_ctr, df_down_ctr, len(up_news), len(down_news))
    phraselst_to_json(phraselst)
    up_news.to_csv(os.path.join(pros_pth, 'filtered_up_ctr.slices'), encoding='utf_8_sig')
    down_news.to_csv(os.path.join(pros_pth, 'filtered_down_slices.csv'), encoding='utf_8_sig')